In [1]:
! pip install pyLDAvis

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk

# Gensim
import gensim
import gensim.corpora as corpora
from nltk import *
from nltk.corpus import stopwords 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis.sklearn


nltk.download('stopwords')
stop_words = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aliis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('data/df_coms_lou_0.csv')
del df['Unnamed: 0']

In [3]:
df

,date,user_id,username,name,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,reply_to,token,n_frasi,n_parole_uniche,n_lettere,n_media_parole,word_f
0,2022-04-12,1150518666771013633,keeleytalks,KeeleyTalks💜,live tonight pm uk pm est summary day 👇,[],1,5,21,"['johnnydepp', 'justiceforjohnnydepp', 'deppvs...",[],"['live', 'tonight', 'pm', 'uk', 'pm', 'est', '...",1,8,39,3.444444,NaN
1,2022-04-12,1150518666771013633,keeleytalks,KeeleyTalks🐝❤️,live tonight pm uk pm est summary day 👇,[],1,5,21,"['johnnydepp', 'justiceforjohnnydepp', 'deppvs...",[],"['live', 'tonight', 'pm', 'uk', 'pm', 'est', '...",1,8,39,3.444444,NaN
2,2022-04-12,1150518666771013633,keeleytalks,KeeleyTalks🐝❤️,live tonight pm uk pm est summary day 👇,[],1,5,21,"['johnnydepp', 'justiceforjohnnydepp', 'deppvs...",[],"['live', 'tonight', 'pm', 'uk', 'pm', 'est', '...",1,8,39,3.444444,NaN
3,2022-04-13,16563753,joleonard___x,𝒥𝑜 . 🤜🏻🥊,issac baruch lady gentleman 😂 👏🏻 shout andysig...,"[{'screen_name': 'andysignore', 'name': 'andy ...",0,0,3,"['deppvsheard', 'popcornedplanet']",[],"['issac', 'baruch', 'lady', 'gentleman', '😂', ...",1,10,71,6.200000,NaN
4,2022-04-15,1150518666771013633,keeleytalks,KeeleyTalks💜,johnny depp v amber heard trial day recapfairf...,[],0,3,9,"['justiceforjohnnydepp', 'johnnydepp', 'deppvs...",[],"['johnny', 'depp', 'v', 'amber', 'heard', 'tri...",1,9,57,5.444444,johnny
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31055,2022-06-07,2535482316,uncivillaw,Uncivil Law 🤠,monica lewinsky join depp heard stupid,[],15,8,73,[],[],"['monica', 'lewinsky', 'join', 'depp', 'heard'...",1,6,38,5.500000,NaN
31056,2022-06-08,1254317649330425857,annieal00836115,Annie Alexander,hoeglaw independent dining depp heard trial we...,[],0,0,1,[],"[{'screen_name': 'HoegLaw', 'name': 'Richard H...","['hoeglaw', 'independent', 'dining', 'depp', '...",1,20,134,5.428571,NaN
31057,2022-06-08,1522266312789020679,paperboats4,violetblue,self proclaimed messiah woman right shame usin...,[],0,1,2,"['amberheard', 'camillevasquez', 'deppvsheard'...",[],"['self', 'proclaimed', 'messiah', 'woman', 'ri...",1,14,92,5.642857,NaN
31058,2022-06-11,377075561,theduiguyplus,The DUI Guy+,h dailymailuk memorabilia depp heard trial sel...,"[{'screen_name': 'dailymailuk', 'name': 'daily...",23,24,403,[],[],"['h', 'dailymailuk', 'memorabilia', 'depp', 'h...",1,11,72,5.636364,NaN


In [4]:
tweets = df.tweet.astype(str).to_list()

In [5]:
tweets

['live tonight pm uk pm est summary day 👇',
 'live tonight pm uk pm est summary day 👇',
 'live tonight pm uk pm est summary day 👇',
 'issac baruch lady gentleman 😂 👏🏻 shout andysignore clip popcornedplanet',
 'johnny depp v amber heard trial day recapfairfax virginia',
 'great job amber team trying prove johnny raging druggie blackout angry violent abuser doctor testified spending majority time placating sick withdrawing drug trying detox',
 'due',
 'great job amber team trying prove johnny raging druggie blackout angry violent abuser doctor testified spending majority time placating sick withdrawing drug trying detox',
 'new video👇',
 'hm turned actually bisexual ok take back everything doubted rekietamedia',
 'rekietamedia trying understand hell weird subtitle trying',
 'sayin… 🤷\u200d♂️ rekietamedia thatumbrella',
 'living milanicosmetics confirming already knew thank milani cosmetic 💕💕💕',
 'living milanicosmetics confirming already knew thank milani cosmetic 💕💕💕',
 'always loved ab

In [6]:
vectorizer_model = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
tweets_vettori = vectorizer_model.fit_transform(tweets)
print(tweets_vettori.shape)

(31060, 3064)


In [7]:
topics_lda = LatentDirichletAllocation(n_components=10, max_iter=5, learning_method='online',verbose=True)
tweets_lda = topics_lda.fit_transform(tweets_vettori)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [8]:
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Argomento %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [9]:
print("Tweets LDA Model:")
selected_topics(topics_lda, vectorizer_model)

Tweets LDA Model:
Argomento 0:
[('make', 977.8272285589667), ('theemilydbaker', 799.3043356843515), ('video', 567.2624327107001), ('trial', 551.2075988630096), ('tmz', 526.8733629056002), ('live', 499.23347893004376), ('watch', 468.62197923750125), ('ianrunkle', 465.34299150533974), ('legalbytesmedia', 454.54817959287254), ('turd', 438.694084449634)]
Argomento 1:
[('expert', 912.20341719029), ('guy', 818.0094917264001), ('shit', 618.3345878684698), ('want', 590.089496881906), ('way', 587.6987170455242), ('man', 561.4472693630341), ('look', 527.5729299377236), ('believe', 400.52117422524225), ('person', 369.53283882643717), ('drug', 353.00084554227396)]
Argomento 2:
[('amber', 1971.016877758774), ('heard', 1219.7332496287763), ('johnny', 932.9394263418079), ('time', 854.595319525133), ('love', 756.93692619185), ('lie', 685.5375882648566), ('friend', 631.9549925024438), ('judge', 592.6228510977539), ('court', 572.6888735784245), ('whitney', 563.717356998665)]
Argomento 3:
[('amber', 801.

C:\Users\aliis\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(topics_lda, tweets_vettori, vectorizer_model, mds='tsne')
dash

C:\Users\aliis\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
C:\Users\aliis\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\aliis\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  12.287858
9       0.648955  71.707100       2        1  11.638975
3      71.019684   9.971006       3        1  10.668911
6       3.785981  33.185135       4        1  10.619500
1     -42.666500  -9.222070       5        1   9.953036
5      -3.621689 -43.526516       6        1   9.901139
8      -3.632402  -4.171708       7        1   9.154558
7     -36.584400  35.804413       8        1   9.123520
0      44.391380  50.615383       9        1   8.777416
4      40.235809 -31.591665      10        1   7.875088, topic_info=         Term         Freq        Total Category  logprob  loglift
3002  witness  1983.000000  1983.000000  Default  30.0000  30.0000
916    elaine  1710.000000  1710.000000  Default  29.0000  29.0000
1288    heard  3595.000000  3595.000000  Default  28.0000  28.0000
408   camille  1205.000000  1205.000000  Default  27.0000  27.0000
1505     jury  1173.000000  1173.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
808    direct    94.593060    95.467661  Topic10  -5.1865   2.5323
2794    today   727.216729   978.344641  Topic10  -3.1469   2.2448
102     amber   583.618271  6229.371389  Topic10  -3.3669   0.1737
1288    heard   360.435762  3595.252050  Topic10  -3.8488   0.2414
1532     knew   119.718842   170.943350  Topic10  -4.9510   2.1853

[424 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3         1  0.996139  absolutely
7         1  0.493789       abuse
7         3  0.270072       abuse
7         4  0.223717       abuse
7         5  0.010077       abuse
...     ...       ...         ...
3045      8  0.996443        yeah
3046      6  0.998700        year
3053     10  0.998894         yes
3054      3  0.997554   yesterday
3059      2  0.998157     youtube

[450 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 10, 4, 7, 2, 6, 9, 8, 1, 5])